# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


## Q1: What is the sum of the outputs of the generator for `limit = 5`?

In [2]:
round(sum(square_root_generator(5)), 3)

8.382

## Q2: What is the 13th number yielded?

In [3]:
# The 13th number is at index 12 since lists start at index 0
round(list(square_root_generator(13))[12], 3)

3.606

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


## Q3

In [5]:
import dlt

pipeline = dlt.pipeline(
    pipeline_name="homework",
    destination="duckdb",
    dataset_name="dlt_homework",
)

# Replace the entire table with data from the first generator on subsequent runs
load_info1 = pipeline.run(
    people_1,
    table_name="people",
    write_disposition="replace",
)
print(load_info1)

# Append data from the second generator
load_info2 = pipeline.run(
    people_2,
    table_name="people",
    write_disposition="append"
)
print(load_info2)

Pipeline homework load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_homework
The duckdb destination used duckdb:////Users/cenvii/data-engineering-zoomcamp-2024/cohorts/2024/workshops/dlt_resources/homework.duckdb location to store data
Load package 1708259112.823832 is LOADED and contains no failed jobs
Pipeline homework load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_homework
The duckdb destination used duckdb:////Users/cenvii/data-engineering-zoomcamp-2024/cohorts/2024/workshops/dlt_resources/homework.duckdb location to store data
Load package 1708259113.396485 is LOADED and contains no failed jobs


In [6]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"set search_path = '{pipeline.dataset_name}'")
print("Loaded tables:")
display(conn.sql("show tables"))

# Check that there are 11 rows
display(conn.sql("select count(*) as row_count from people"))

# Calculate sum of all ages
display(conn.sql("select sum(age) as total_age from people"))

Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merged       │
└─────────────────────┘

┌───────────┐
│ row_count │
│   int64   │
├───────────┤
│        11 │
└───────────┘

┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       353 │
└───────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


## Q4

In [7]:
import dlt

pipeline = dlt.pipeline(
    pipeline_name="homework",
    destination="duckdb",
    dataset_name="dlt_homework",
)

# Replace the entire table with data from the first generator on subsequent runs
load_info1 = pipeline.run(
    people_1,
    table_name="people_merged",
    write_disposition="replace",
    primary_key="ID",
)
print(load_info1)

# Merge data from the second generator with the data above using `ID` as primary key
load_info2 = pipeline.run(
    people_2,
    table_name="people_merged",
    write_disposition="merge",
    primary_key="ID",
)
print(load_info2)

Pipeline homework load step completed in 0.14 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_homework
The duckdb destination used duckdb:////Users/cenvii/data-engineering-zoomcamp-2024/cohorts/2024/workshops/dlt_resources/homework.duckdb location to store data
Load package 1708259113.971963 is LOADED and contains no failed jobs
Pipeline homework load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_homework
The duckdb destination used duckdb:////Users/cenvii/data-engineering-zoomcamp-2024/cohorts/2024/workshops/dlt_resources/homework.duckdb location to store data
Load package 1708259114.471505 is LOADED and contains no failed jobs


In [8]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"set search_path = '{pipeline.dataset_name}'")
print("Loaded tables:")
display(conn.sql("show tables"))

# Check that there are 8 rows after merging
display(conn.sql("select count(*) as row_count from people_merged"))

# Check that ID 3 has age 33
display(conn.sql("select age from people_merged where id = 3"))

# Calculate sum of all ages
display(conn.sql("select sum(age) as total_age from people_merged"))

Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merged       │
└─────────────────────┘

┌───────────┐
│ row_count │
│   int64   │
├───────────┤
│         8 │
└───────────┘

┌───────┐
│  age  │
│ int64 │
├───────┤
│    33 │
└───────┘

┌───────────┐
│ total_age │
│  int128   │
├───────────┤
│       266 │
└───────────┘

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
# %%capture
# %pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX